In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import peakutils


In [2]:
#Upload the .txt file with the analysed spiketrains
df = pd.read_csv('path/file_name.txt', encoding='UTF-16', delimiter='\t') #change path for uploading file

#clean columns/adjust names
df = df.drop(["Name"], axis=1)
df.columns = df.columns.str.replace('#', 'Roi_').str.replace(' (New)', '')

In [4]:
#Drop NaN values
df = df.dropna(axis=1, how='all')

In [ ]:
#Choose the degree and the iterations for the polinomial fit
deg = 5
max_it = 250

# Initialize an empty dictionary to store the mean baseline values for each ROI
baseline_means = {}

# Iterate over all columns except the first two
for col in df.columns[2:]:
    # Extract the column as a NumPy array
    y = df[col].values

    # Compute the baseline
    bl = peakutils.baseline(y, deg=deg, max_it=max_it)

    # Plot the original data and the baseline on the same plot
    plt.figure()
    sns.lineplot(x=df["Time [s]"], y=y, label='Original')
    sns.lineplot(x=df["Time [s]"], y=bl, label='Baseline')
    plt.title(f"Original and Baseline for {col}")
    plt.legend()
    plt.show()

    # Subtract the baseline from the original data
    corrected = y - bl

    # Store the corrected data back in the DataFrame
    df[col] = corrected

    # Compute the mean of the corrected baseline
    baseline_mean = np.mean(bl)
    baseline_means[col] = baseline_mean  # Store this for normalization

    # Plot the corrected data
    plt.figure()
    sns.lineplot(x=df["Time [s]"], y=corrected, label='Corrected')
    plt.title(f"Corrected data for {col}")
    plt.show()

# Now compute ΔF/F0 using the stored mean baseline values for each ROI
for col in df.columns[2:]:
    df[col] = df[col] / baseline_means[col]

# Reconstruct the DataFrame with normalized data
df_norm = pd.concat([df.iloc[:, :2], df.iloc[:, 2:]], axis=1)

#save the dataframe
df_norm.to_csv('path/file_name_analysis.csv', index=False)  #chnage path


In [ ]:
#Check ther normalised traces: 
if 'df_norm' in locals():
    pass
else:
    df_norm = pd.read_csv('path/file_name_analysis.csv') # Change path

# max value of intensity among the df
max_value = df_norm.iloc[:, 2:].max().max()
max_value
fig = go.Figure()



for column in df_norm.iloc[:, 2:].columns:
    fig.add_trace(go.Scatter(x=df_norm['Time [s]'], y=df_norm[column], mode='lines', name=f'{column}'))

fig.update_layout(
    xaxis=dict(title='Time (s)', dtick=250, tickformat='d'),
    yaxis_title="Normalized Intensity",
    title="Normalized Intensity over Time",
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02
    ),
    title_x=0.5,  # Title centered above the graph
    title_y=0.9,   # Adjust title position above the graph
    height=600,
    width=800,
    showlegend=True,
    hovermode='x'
)

fig.show()